## OpenAI Embedding
* api 키를 발급받아 사용

In [4]:
import sys
import os

sys.path.append(os.path.abspath("../../data/API_KEY"))

In [5]:
from openai import OpenAI
import OpenAI_API_Key

API_KEY = OpenAI_API_Key.API_KEY

client = OpenAI(api_key=API_KEY)

In [15]:
embedding = client.embeddings.create(
    model="text-embedding-ada-002",
    input=["야 저기 차 온다!", "저건 어디로 가는 버스야?"],
)

len(embedding.data[0].embedding)

1536

In [7]:
print(type(embedding))  # OpenAI API의 임베딩 요청에 대한 응답
print(type(embedding.data[0]))  # 각 문장의 임베딩 벡터에 대한 정보를 담은 객체
print(type(embedding.data[0].embedding))  # 임베딩 벡터 (list)

<class 'openai.types.create_embedding_response.CreateEmbeddingResponse'>
<class 'openai.types.embedding.Embedding'>
<class 'list'>


In [17]:
import pandas as pd

data = [
    "내일 차 타고 놀러가자",
    "지금 오는 버스는 어디서 오는 버스야?",
    "차 한잔 하면서 이야기 하시죠",
    "5차 공동구매! 오늘만 세일!",
    "홍차 녹차 중에 어떤 차가 좋으세요?",
]

df = pd.DataFrame(data, columns=["text"])
df

,text
0,내일 차 타고 놀러가자
1,지금 오는 버스는 어디서 오는 버스야?
2,차 한잔 하면서 이야기 하시죠
3,5차 공동구매! 오늘만 세일!
4,홍차 녹차 중에 어떤 차가 좋으세요?


In [18]:
def get_embedding(text):
    response = client.embeddings.create(input=text, model="text-embedding-ada-002")
    return response.data[0].embedding

In [19]:
df["embedding"] = [get_embedding(text) for text in df["text"]]
df

,text,embedding
0,내일 차 타고 놀러가자,"[0.015895135700702667, -0.011393633671104908, ..."
1,지금 오는 버스는 어디서 오는 버스야?,"[0.008037855848670006, -0.011817412450909615, ..."
2,차 한잔 하면서 이야기 하시죠,"[0.016478225588798523, -0.02388259395956993, 0..."
3,5차 공동구매! 오늘만 세일!,"[-0.017510101199150085, -0.002525998977944255,..."
4,홍차 녹차 중에 어떤 차가 좋으세요?,"[0.005494758952409029, -0.011675575748085976, ..."


## 유사도 구하기

In [27]:
import numpy as np


def cos_sim(A, B):
    return A @ B / (np.linalg.norm(A) * np.linalg.norm(B))


def return_answer_candidate(df, query):
    """
    df : 문장과 임베딩 벡터가 들어있는 데이터 프레임
    query : 질의할 문장
    """
    query_embedding = get_embedding(query)

    df["similarity"] = [
        cos_sim(np.array(emb_vec), np.array(query_embedding))
        for emb_vec in df["embedding"]
    ]

    results_co = df.sort_values("similarity", ascending=False, ignore_index=True)
    return results_co.head(3)

In [28]:
sim_result = return_answer_candidate(df, "야 저기 차 온다")
sim_result

,text,embedding,similarity
0,내일 차 타고 놀러가자,"[0.015895135700702667, -0.011393633671104908, ...",0.896481
1,차 한잔 하면서 이야기 하시죠,"[0.016478225588798523, -0.02388259395956993, 0...",0.875385
2,지금 오는 버스는 어디서 오는 버스야?,"[0.008037855848670006, -0.011817412450909615, ...",0.848097


In [29]:
sim_result = return_answer_candidate(df, "예쁜 카페가고 싶어")
sim_result

,text,embedding,similarity
0,내일 차 타고 놀러가자,"[0.015895135700702667, -0.011393633671104908, ...",0.822717
1,차 한잔 하면서 이야기 하시죠,"[0.016478225588798523, -0.02388259395956993, 0...",0.812945
2,홍차 녹차 중에 어떤 차가 좋으세요?,"[0.005494758952409029, -0.011675575748085976, ...",0.812300
